In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)


In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'EPSG:32638'
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/ET_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
#read in username and password
# exec(open('/Users/wanders7/Documents/Code/Logins/FDW.py').read())
auth = ('cxuan', 'cx941207')
parameters = {
    'format': 'json',
    'country': 'Ethiopia',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={},verify=False)
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

ValueError: No objects to concatenate

In [ ]:

# Manual Pre-processing before Sweeping --------- #
# 1. Default setting
# a.1) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# a.2) Early in the record the catch-all crop production system is 'all' rather than none. Make these to none-type
df.loc[df['crop_production_system'] == 'all', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# ----------------------------------------------- #
#Drop the hundreds of blank (nan) rows that are duplicates of rows that do have data
df=df[np.isfinite(df.value)]
df=df.dropna(subset=['admin_2'])
rows = ['fnid','season_year', 'product','indicator','crop_production_system']
#find duplicates then aggregate to fnid after pivoting around the indicator
# dupes = df[rows[:-1]][df[rows[:-1]].duplicated(keep=False)]
# dupes.to_csv('C:/Users/chenxuan/Desktop/dupes_ET.csv')
# dupes_raw=df.loc[dupes.index]
# dupes_raw.to_csv('C:/Users/chenxuan/Desktop/dupes_raw_ET.csv')
# df['collection_date'] = pd.to_datetime(df['collection_date'])
# df = df.sort_values(by=['collection_date']).drop_duplicates(rows, keep='last').reset_index(drop=True)

# dupes = dupes.groupby(by=['fnid','season_year','indicator','crop_production_system','product']).sum()
# dupes.reset_index(inplace=True)
# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# # FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([ET_Admin2_1994,ET_Admin2_2001,ET_Admin2_2003,ET_Admin2_2007,ET_Admin2_2008,ET_Admin2_2014,ET_Admin2_2019,ET_Admin2_2020,ET_Admin2_2021], axis=0)
PlotAdminShapes(shape_used, label=True)
# # ----------------------------------------------- #


In [ ]:
sorted(list(df_origin['season_year'].unique()))

# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = ET_Admin2_2014.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [ ]:
# Link admin boundaries ------------------------- #
link_1994, over_1994 = FDW_PD_CreateAdminLink(ET_Admin2_1994, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2001, over_2001 = FDW_PD_CreateAdminLink(ET_Admin2_2001, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2003, over_2003 = FDW_PD_CreateAdminLink(ET_Admin2_2003, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2007, over_2007 = FDW_PD_CreateAdminLink(ET_Admin2_2007, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2008, over_2008 = FDW_PD_CreateAdminLink(ET_Admin2_2008, ET_Admin2_2014, 'ADMIN2', 'ADMIN2', prod, epsg)

# Crop specific ratios
link_ratio_1994 = FDW_PD_RatioAdminLink(link_1994, prod, over_1994, mdx_pss)
link_ratio_2001 = FDW_PD_RatioAdminLink(link_2001, prod, over_2001, mdx_pss)
link_ratio_2003 = FDW_PD_RatioAdminLink(link_2003, prod, over_2003, mdx_pss)
link_ratio_2007 = FDW_PD_RatioAdminLink(link_2007, prod, over_2007, mdx_pss)
link_ratio_2008 = FDW_PD_RatioAdminLink(link_2008, prod, over_2008, mdx_pss)

# Merge link_ratio
assert link_ratio_1994.keys() == link_ratio_2008.keys()
assert link_ratio_2001.keys() == link_ratio_2008.keys()
assert link_ratio_2003.keys() == link_ratio_2008.keys()
assert link_ratio_2007.keys() == link_ratio_2008.keys()
link_merged = [link_ratio_1994, link_ratio_2001,link_ratio_2003,link_ratio_2007]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #


In [ ]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "growing year"
cols = ['season_name','product','crop_production_system','growing_month','harvest_year','harvest_month','growing_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','growing_year','growing_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack_gscd = stack[names]
stack_gscd['gscd_code'] = 'calibrated'
# ----------------------------------------------- #
# Reported FDW data ----------------------------- #
stack_fdw = df[names]
stack_fdw['indicator'] = stack_fdw['indicator'].replace({'Area Harvested':'area','Quantity Produced':'production','Yield':'yield'})
stack_fdw['gscd_code'] = 'reported'
# ----------------------------------------------- #
# Final Processing ------------------------------ #
stack = pd.concat([stack_fdw,stack_gscd], axis=0).reset_index(drop=True)
# No concerns found for grain types
stack['product'] = stack['product'].replace(product_category)
# Calibration of Growing and Harvest year and season
cs = {
    'Meher': {'growing_month':{'10-01':'10-01'},'harvest_month':{'01-01':'01-01'}},
}
cy = {}
stack, df, link_ratio = FDW_PD_CaliSeasonYear(stack, df, link_ratio, cs, cy)
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# # ----------------------------------------------- #

# Save data
save_hdf('./data/crop/adm_crop_production_ET.hdf', stack)
save_hdf('./data/crop/adm_crop_production_ET_raw.hdf', df)
save_npz('./data/crop/adm_crop_production_ET_ratio.npz', link_ratio)


In [ ]:
stack_gscd

In [17]:
# Bar chart of national grain production
country_iso, country_name = 'ET', 'Ethiopia'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Maize', 'Wheat','Millet', 'Teff','Sorghum','Barley','Oats','Rice']
for season_name in ['Meher']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = './figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()


./figures/ET_bar_natgrainprod_Meher.png is saved.


In [18]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'ET', 'Ethiopia'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Barley', 'Meher'],
    ['Maize', 'Meher'],
    ['Millet', 'Meher'],
    ['Teff', 'Meher'],
    ['Sorghum', 'Meher'],
    ['Wheat', 'Meher'],
    ['Rice', 'Meher'],
    ['Oats', 'Meher'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = './figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

./figures/ET_line_pay_Barley_Meher.png is saved.


./figures/ET_line_pay_Maize_Meher.png is saved.


./figures/ET_line_pay_Millet_Meher.png is saved.


./figures/ET_line_pay_Teff_Meher.png is saved.


./figures/ET_line_pay_Sorghum_Meher.png is saved.


./figures/ET_line_pay_Wheat_Meher.png is saved.


./figures/ET_line_pay_Rice_Meher.png is saved.


./figures/ET_line_pay_Oats_Meher.png is saved.


In [19]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'ET', 'Ethiopia'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
link_ratio = load_npz('./data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
product_season = [
    ['Barley', 'Meher'],
    ['Maize', 'Meher'],
    ['Millet', 'Meher'],
    ['Teff', 'Meher'],
    ['Sorghum', 'Meher'],
    ['Wheat', 'Meher'],
    ['Rice', 'Meher'],
    ['Oats', 'Meher'],
]
for product_name, season_name in product_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = './figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)


./figures/crop_calibrated/ET_Barley_Meher_ET2014A20101.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20102.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20103.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20104.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20105.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20301.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20302.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20303.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20304.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20305.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20306.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20307.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20308.png is saved.
./figures/crop_calibrated/ET_Barley_Meher_ET2014A20309.png is saved.
./figures/crop_calibrated/ET_Barle

./figures/crop_calibrated/ET_Maize_Meher_ET2014A20606.png is saved.
./figures/crop_calibrated/ET_Maize_Meher_ET2014A20501.png is saved.
./figures/crop_calibrated/ET_Maize_Meher_ET2014A20509.png is saved.
./figures/crop_calibrated/ET_Maize_Meher_ET2014A20605.png is saved.
./figures/crop_calibrated/ET_Maize_Meher_ET2014A21501.png is saved.
./figures/crop_calibrated/ET_Maize_Meher_ET2014A20201.png is saved.
./figures/crop_calibrated/ET_Maize_Meher_ET2014A21202.png is saved.
./figures/crop_calibrated/ET_Maize_Meher_ET2014A21204.png is saved.
./figures/crop_calibrated/ET_Millet_Meher_ET2014A20101.png is saved.
./figures/crop_calibrated/ET_Millet_Meher_ET2014A20102.png is saved.
./figures/crop_calibrated/ET_Millet_Meher_ET2014A20103.png is saved.
./figures/crop_calibrated/ET_Millet_Meher_ET2014A20104.png is saved.
./figures/crop_calibrated/ET_Millet_Meher_ET2014A20105.png is saved.
./figures/crop_calibrated/ET_Millet_Meher_ET2014A20301.png is saved.
./figures/crop_calibrated/ET_Millet_Meher_

./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20306.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20307.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20308.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20309.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20310.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20312.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20401.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20402.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20403.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20404.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20405.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20407.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20408.png is saved.
./figures/crop_calibrated/ET_Sorghum_Meher_ET2014A20409.png is saved.
./figures/crop_calib

./figures/crop_calibrated/ET_Rice_Meher_ET2014A20403.png is saved.
./figures/crop_calibrated/ET_Rice_Meher_ET2014A20404.png is saved.
./figures/crop_calibrated/ET_Rice_Meher_ET2014A20419.png is saved.
./figures/crop_calibrated/ET_Rice_Meher_ET2014A20602.png is saved.
./figures/crop_calibrated/ET_Rice_Meher_ET2014A20704.png is saved.
./figures/crop_calibrated/ET_Rice_Meher_ET2014A21201.png is saved.
./figures/crop_calibrated/ET_Rice_Meher_ET2014A21203.png is saved.
./figures/crop_calibrated/ET_Rice_Meher_ET2014A20605.png is saved.
./figures/crop_calibrated/ET_Oats_Meher_ET2014A20301.png is saved.
./figures/crop_calibrated/ET_Oats_Meher_ET2014A20302.png is saved.
./figures/crop_calibrated/ET_Oats_Meher_ET2014A20303.png is saved.
./figures/crop_calibrated/ET_Oats_Meher_ET2014A20304.png is saved.
./figures/crop_calibrated/ET_Oats_Meher_ET2014A20305.png is saved.
./figures/crop_calibrated/ET_Oats_Meher_ET2014A20306.png is saved.
./figures/crop_calibrated/ET_Oats_Meher_ET2014A20307.png is sa